In [5]:
import glob
import os
import re
import cv2
import json
import csv

In [6]:
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(data, key=alphanum_key)

In [13]:
scorer = 'ymc'
body_parts = ['leg1', 'leg2', 'leg3', 'leg4', 'leg5', 'leg6', 'head', 'tail', 'prob']
folder_path = 'label/1'
video_name = 'video1'

In [14]:
data_folder = os.path.join('labeled-data', video_name)
os.makedirs(data_folder)
csv_path = os.path.join(data_folder, f'CollectedData_{scorer}.csv')

video_folder = 'videos'
if not os.path.exists(video_folder):
    os.makedirs(video_folder)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
example_img = cv2.imread(glob.glob(os.path.join(folder_path, '*.jpg'))[0])
h, w = example_img.shape[:2]
video_writer = cv2.VideoWriter(f'{os.path.join(video_folder, video_name)}.avi', fourcc, 30, (w, h))

with open(csv_path, 'w+', newline='') as f:
    csv_writer = csv.writer(f)
    num_kpts = len(body_parts)
    csv_writer.writerow(['scorer'] + [scorer] * 2 * num_kpts)
    csv_writer.writerow(['bodyparts'] + [body_parts[i//2] for i in range(2 * num_kpts)])
    csv_writer.writerow(['coords'] + ['x', 'y'] * num_kpts)

    img_paths = sorted_alphanumeric(glob.glob(os.path.join(folder_path, '*.jpg')))
    json_paths = sorted_alphanumeric(glob.glob(os.path.join(folder_path, '*.json')))
    num_imgs = len(img_paths)

    for i in range(num_imgs):
        data = json.load(open(json_paths[i], 'r'))['shapes']
        row = ['', ''] * num_kpts
        for kpt in data:
            kpt_idx = int(kpt['label'])
            kpt_coord = kpt['points']
            row[2*(kpt_idx-1) : 2*kpt_idx] = kpt_coord[0]

        new_img_path = os.path.join(data_folder, f'{i+1}.jpg')
        row = [new_img_path] + row
        csv_writer.writerow(row)

        img = cv2.imread(img_paths[i])
        cv2.imwrite(new_img_path, img)
        video_writer.write(img)

video_writer.release()